In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Tikhonov regularisation using CGLS and block framework
Few lines intro

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularization parameter and operator in regulariser.

In [ ]:
#imports
from ccpi.framework import ImageGeometry, ImageData 
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import BlockDataContainer
from ccpi.framework import TestData

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity

from ccpi.astra.operators import AstraProjectorSimple 
from ccpi.optimisation.operators import FiniteDiff

import tomophantom
from tomophantom import TomoP2D
import scipy
import numpy as np                          
import matplotlib.pyplot as plt

import os

In [ ]:
def plot2D_2(datacontainer1, title1, datacontainer2, title2):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,15))
    plt.subplots_adjust(wspace = 0.5)
    
    ax1.set_title(title1)
    subplot1 = ax1.imshow(datacontainer1.as_array())
    plt.colorbar(subplot1, ax=ax1,fraction=0.0467, pad=0.02)

    ax2.set_title(title2)
    subplot2 = ax2.imshow(datacontainer2.as_array())
    plt.colorbar(subplot2, ax=ax2,fraction=0.0467, pad=0.02)
        
    plt.show()
    

In [ ]:
def plot2D_1(datacontainer1, title1):
    plt.imshow(datacontainer1.as_array())
    plt.title(title1)
    plt.figsize=(15,15)
    plt.colorbar(fraction=0.032, pad=0.02)    
    plt.show()

In [ ]:
def plot2D_1_array(array, title1):
    plt.imshow(array)
    plt.title(title1)
    plt.figsize=(15,15)
    plt.colorbar(fraction=0.032, pad=0.02)    
    plt.show()

### Setting up the dataset - 2D

In [ ]:
#set up acquisition geometry
number_pixels_x = 1024
number_projections = 180
angles = np.linspace(0, np.pi, number_projections, dtype=np.float32)
ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)

#set up image geometry
num_voxels_xy = 1024
ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)

In [ ]:
# Load Shepp-Logan phantom 
model = 1
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

#tomophantom takes angular input in degrees
phantom_2D = TomoP2D.Model(model, num_voxels_xy, path_library2D)
phantom_sino = TomoP2D.ModelSino(model, num_voxels_xy, number_pixels_x, angles*180./np.pi, path_library2D)

#rescale the tomophantom data, set the max absoobtion to 25%
set_ratio_absorption = 0.25
new_max_value = -np.log(set_ratio_absorption)
sino_max = np.amax(phantom_sino)
scale = new_max_value/sino_max

#allocate the image data container and copy the dataset in
#this is only used as a reference to the gorund truth
model = ig.allocate(0)
model.fill(phantom_2D*scale)

#allocate the acquisition data container and copy the sinogram in
sinogram = ag.allocate(0)
sinogram.fill(phantom_sino*scale)

In [ ]:
#add poisson noise to the sinogram
background_counts = 100 #lower counts will increase the noise
counts = background_counts * np.exp(-sinogram.as_array())
noisy_counts = np.random.poisson(counts)
sino_out = -np.log(noisy_counts/background_counts)

In [ ]:
sinogram_noisy = ag.allocate()
sinogram_noisy.fill(sino_out)
plot2D_2(sinogram, "sinogram", sinogram_noisy, "np sinogram noisy")

<a id="section_CGLS_simple"></a>
### Reconstruct using unregularised CGLS

Solve:
$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}A u - b\end{Vmatrix}^2_2$$

where,

- $A$ is the projection operator

- $b$ is the acquired data

- $u$ is the solution



In [ ]:
#define the operator A
device = "gpu"
operator = AstraProjectorSimple(ig, ag, device)

In [ ]:
#define the data b
data = sinogram_noisy

In [ ]:
#setup CGLS
x_init = ig.allocate(0)
cgls = CGLS(x_init=x_init, operator=operator, data=data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#Run N interations
cgls.run(100, verbose = True)

In [ ]:
#plot the results
CGLS_simple = cgls.get_output()

residuals = ig.allocate(0)
residuals.fill(cgls.get_output().as_array()- model.as_array())
plot2D_2(CGLS_simple, "CGLS reconstruction", residuals, "Difference from ground truth")

<span style="color:red;font-size:larger">**Exercise 1:**</span> Repeat this with the noisy dataset. Remember you can change the number of iteations to run between outputs. [go to section start](#section_CGLS_simple)

### Reconstruct using regularised CGLS

#### Regularisation

Noisy datasets lead to an ill-posed problem. If we try to solve these using LS we end up with a noisy reconstruction. Regularisation adds information in order for us to solve the problem.

#### CGLS and regularisation

Adding a differentiable regulariser...

Identity

gradient



Solve:
$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}A u - b \end{Vmatrix}^2_2 + \alpha^2\|Lu\|^2_2$$


where,

- $A$ is the projection operator

- $b$ is the acquired data

- $u$ is the solution

- $\alpha$ is the regularisation parameter

- $L$ is a regularisation operator

<br>This can be re-written in the form:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\binom{A}{\alpha L} u - \binom{b}{0}\end{Vmatrix}^2_2$$

Which allows us to solve it using CGLS in the form:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}\tilde{A} u - \tilde{b}\end{Vmatrix}^2_2$$

where:

- $\tilde{A} = \binom{A}{\alpha L}$

- $\tilde{b} = \binom{b}{0}$



#### Introducing the block framework

We can construct $\tilde{A}$ and $\tilde{b}$ using the BlockFramework in the CIL.

$\tilde{A}$ is a BlockOperator

`BlockOperator(op0,op1,shape=(1,2))` results in a row block


`BlockOperator(op0,op1)`
and
`BlockOperator(op0,op1,shape=(2,1))` result in a column block

$\tilde{b}$ is a BlockDataContainer

`BlockDataContainer(DataContainer0, DataContainer1)`


<a id="section_CGLS_alpha"></a>
#### Reconstruct using CGLS and the identity operator

The identity opeator blah blah

<span style="color:red;font-size:larger">**Exercise #:**</span> Construct the BlockOperator $\tilde{A}$ using the idenity operator

In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Identity(ig)
alpha = 15

#operator_block = BlockOperator(  )
operator_block = BlockOperator( A, alpha * L)

In [ ]:
#define the data b
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate(0))

Run CGLS as before, but passing the BlockOperator and BlockDataContainer

In [ ]:
#setup CGLS with the Block Operator and Block DataContainer
x_init = ig.allocate(0)      
cgls = CGLS(x_init=x_init, operator=operator_block, data=data_block)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(100, verbose = True)

In [ ]:
CGLS_regularised = cgls.get_output()

residuals = ig.allocate(0)
residuals.fill(cgls.get_output().as_array()- model.as_array())
plot2D_2(CGLS_regularised, "CGLS reconstruction", residuals, "Difference from ground truth")

<span style="color:red;font-size:larger">**Exercise #:**</span> Repeat this with different regularisation weights [go to section start](#section_CGLS_alpha)

### A more detailed look at the BlockFramework

#### BlockDataContainer 

BlockDataContainer holds datacontainers as a column vector

$$x = [x_{1}, x_{2} ]\in (X_{1}\times X_{2})$$
$$y = [y_{1}, y_{2}, y_{3} ]\in(Y_{1}\times Y_{2} \times Y_{3})$$


#### BlockOperator: 

A Block matrix with operators 

$$ K = \begin{bmatrix}
A_{1} & A_{2} \\
A_{3} & A_{4} \\
A_{5} & A_{6}
\end{bmatrix}_{(3,2)} *  \quad \underbrace{\begin{bmatrix}
x_{1} \\
x_{2} 
\end{bmatrix}_{(2,1)}}_{\textbf{x}} =  \begin{bmatrix}
A_{1}x_{1}  + A_{2}x_{2}\\
A_{3}x_{1}  + A_{4}x_{2}\\
A_{5}x_{1}  + A_{6}x_{2}\\
\end{bmatrix}_{(3,1)} =  \begin{bmatrix}
y_{1}\\
y_{2}\\
y_{3}
\end{bmatrix}_{(3,1)} = \textbf{y}$$

Column: Share the same domains $X_{1}, X_{2}$<br>
Rows: Share the same ranges $Y_{1}, Y_{2}, Y_{3}$

$$ K : (X_{1}\times X_{2}) \rightarrow (Y_{1}\times Y_{2} \times Y_{3})$$


$$ A_{1}, A_{3}, A_{5}: \text{share the same domain }  X_{1}$$
$$ A_{2}, A_{4}, A_{6}: \text{share the same domain }  X_{2}$$

$$A_{1}: X_{1} \rightarrow Y_{1}, \quad A_{3}: X_{1} \rightarrow Y_{2}, \quad  A_{5}: X_{1} \rightarrow Y_{3}$$
$$A_{2}: X_{2} \rightarrow Y_{1}, \quad A_{4}: X_{2} \rightarrow Y_{2}, \quad  A_{6}: X_{2} \rightarrow Y_{3}$$

##### An example: The gradient operator

The gradient operator uses BlockDataContainers. ??ToDo: implement gradient as block of FD? or just leave as it is?

The direct gradient operator $\nabla$ acts on an image $u$ and returns a BlockDataContainer $\textbf{y}$

$$ \nabla(u) = 
\begin{bmatrix}
   \nabla_x\\
   \nabla_y\\
\end{bmatrix}
*u =
\begin{bmatrix}
    \nabla_xu\\
    \nabla_yu\\
\end{bmatrix}
=  
\begin{bmatrix}w_{x}\\w_{y}\end{bmatrix}= \textbf{w}$$

The adjoint gradient operator $\nabla^*$ acts on the BlockDataContainer $\textbf{y}$ and returns an image $\rho$

$$  \nabla^*(\textbf w) = 
\begin{bmatrix}
    \nabla^*_x &
    \nabla^*_y
\end{bmatrix}
*
\begin{bmatrix}
    w_{x}\\
    w_{y}\\
\end{bmatrix} 
=
\begin{bmatrix}
    \nabla^*_x w_x + \nabla^*_y w_y
\end{bmatrix} =  \rho$$

In [ ]:
#loading new test image
loader = TestData()
shapes = loader.load(TestData.SHAPES)
shapes_ig = shapes.geometry

#plot ths results
plot2D_1(shapes, "shapes")

Define the gradient as the FD

Introduce the Finite Difference opearator

`from ccpi.optimisation.operators import FiniteDiff`

set it up in x and y

Setting up and running the FD operator

In [ ]:
#define the operator FiniteDiff - needs to image geometry, the direction and the boundary conditions
fdx = FiniteDiff(shapes_ig, direction=1, bnd_cond='Neumann')

#run it over the input image
image_2D_dx = fdx.direct(shapes)

#plot ths results
plot2D_1(image_2D_dx, "dx")

<span style="color:red;font-size:larger">**Exercise #:**</span> Create a blockOperator to calculate the finite difference in x and y

In [ ]:
#difine the directions and boundary conditions
fdx = FiniteDiff(shapes_ig, direction=1, bnd_cond='Neumann')
fdy = FiniteDiff(shapes_ig, direction=0, bnd_cond='Neumann')

In [ ]:
#comstruct the block operator
FD = BlockOperator(fdx, fdy)

In [ ]:
#run it on the test image
fd_out = FD.direct(shapes)

In [ ]:
#plot the results
plot2D_2(fd_out.get_item(0), "dx", fd_out.get_item(1), "dy")

A closer look at data types

In [ ]:
#input is ImageData
print("Input")
print("\ttype:\t", type(shapes))
print("\tshape:\t", shapes.shape)

In [ ]:
#output is BloackDataContainer
print("Output")
print("\ttype:\t", type(fd_out))
print("\tshape:\t", fd_out.shape)

print("\tDataContainer 0")
print("\t\ttype:\t", type(fd_out.get_item(0)))
print("\t\tshape:\t", fd_out.get_item(0).shape)

print("\tDataContainer 1")
print("\t\ttype:\t", type(fd_out.get_item(1)))
print("\t\tshape:\t", fd_out.get_item(1).shape)

In [ ]:
#split containers
dx = fd_out.get_item(0)
dy = fd_out.get_item(1)

#calculate the squared norm of the x and y gradients
dx2 = (dx**2).sum()
dy2 = (dy**2).sum()
sq_norm = dx2 + dy2
print(sq_norm)

In [ ]:
#do the same thing within the blockframework
sq_norm = fd_out.squared_norm()
print(sq_norm)

The BlockFramework provides basic algebra between BlockDataContainers, DataContainers, subclasses and scalars providing the shape of the containers are compatible
- add
- subtract
- multiply
- divide
- power
- squared_norm

<span style="color:red;font-size:larger">**Exercise #:**</span> Run the adjoint operator. What data type does it take in and write out?


In [ ]:
#run the adjoint method
adjoint_output = FD.adjoint(fd_out)

plot2D_1(adjoint_output, "adjoint gradient")

print(adjoint_output.shape)

### Reconstruct using regularised CGLS with Tikhonov regularisation

#### Tikhonov regularisation

$$ \underset{x}{\mathrm{argmin}}\begin{Vmatrix}A u - b \end{Vmatrix}^2_2 + \alpha^2\begin{Vmatrix}\nabla u\end{Vmatrix}^2_2$$

For this use the gradient operator. And optimised form of FD over the space or space+time dimensions.

In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Gradient(ig)
alpha = 50

operator_block = BlockOperator( A, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate(0))

In [ ]:
#setup CGLS with the block operator and block data
x_init = ig.allocate(0)      
cgls = CGLS(x_init=x_init, operator=operator_block, data=data_block)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(100, verbose = True)

In [ ]:
CGLS_Tikhonov = cgls.get_output()

residuals = ig.allocate(0)
residuals.fill(cgls.get_output().as_array()- model.as_array())
plot2D_2(CGLS_Tikhonov, "CGLS reconstruction", residuals, "Difference from ground truth")


### Summary

In [ ]:
#compare the outputs of unregularise and regularised CGLS
plot2D_2(model, "Ground truth", CGLS_simple, "CGLS simple")
plot2D_2(CGLS_regularised, "CGLS with Idenity regularisation", CGLS_Tikhonov, "CGLS with Tikhonov regularisation")